<a href="https://colab.research.google.com/github/igorgcgv/An-lise-de-Dados-no-Ramo-da-Educa-o/blob/main/Projeto_de_Limpeza_de_Dados_de_Telecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando as Libs


In [ ]:


import math
import sys, os
import numpy as np
import pandas as pd


### Carregando as Funções Utilizadas nesse Projeto

 Módulo de limpeza e tratamento de valores ausentes

In [ ]:

# Calcula o percentual de valores ausentes
def func_calc_percentual_valores_ausentes(df):

    # Calcula o total de células no dataset
    totalCells = np.product(df.shape)

    # Conta o número de valores ausentes por coluna
    missingCount = df.isnull().sum()

    # Calcula o total de valores ausentes
    totalMissing = missingCount.sum()

    # Calcula o percentual de valores ausentes
    print("O dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")


# Função que calcula o percentual de linhas com valores ausentes
def func_calc_percentual_valores_ausentes_linha(df):

    # Calcula o número total de linhas com valores ausentes
    missing_rows = sum([True for idx,row in df.iterrows() if any(row.isna())])

    # Calcula o número total de linhas
    total_rows = df.shape[0]

    # Calcula a porcentagem de linhas ausentes
    print(round(((missing_rows/total_rows) * 100), 2), "%", "das linhas no conjunto de dados contêm pelo menos um valor ausente.")


# Função para calcular valores ausentes por coluna
def func_calc_percentual_valores_ausentes_coluna(df):
    
    # Total de valores ausentes
    mis_val = df.isnull().sum()

    # Porcentagem de valores ausentes
    mis_val_percent = 100 * mis_val / len(df)

    # Tipo de dado das colunas com valores ausentes
    mis_val_dtype = df.dtypes

    # Cria uma tabela com os resultados
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

    # Renomear as colunas
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Valores Ausentes', 1 : '% de Valores Ausentes', 2: 'Dtype'})

    # Classifica a tabela por porcentagem de valores ausentes de forma decrescente e remove colunas sem valores faltantes
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)

    # Print 
    print ("O dataset tem " + str(df.shape[1]) + " colunas.\n"
        "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")

    if mis_val_table_ren_columns.shape[0] == 0:
        return

    # Retorna o dataframe com informações ausentes
    return mis_val_table_ren_columns


# Imputação de valores ausentes usando forward fill (preenchimento progressivo)
# method = 'ffill': Ffill ou forward-fill propaga o último valor não nulo observado para frente até que outro valor não nulo seja encontrado
def fix_missing_ffill(df, col):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(method = 'ffill')
    print(f"{count} valores ausentes na coluna {col} foram substituídos usando o método de preenchimento progressivo.")
    return df[col]


# Imputação de valores ausentes usando backward fill
# method = 'bfill': Bfill ou backward-fill propaga o primeiro valor não nulo observado para trás até que outro valor não nulo seja encontrado
def fix_missing_bfill(df, col):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(method = 'bfill')
    print(f"{count} valores ausentes na coluna {col} foram substituídos usando o método de preenchimento reverso.")
    return df[col]


# Imputação usando a mediana
def fix_missing_median(df, col):
    median = df[col].median()
    count = df[col].isna().sum()
    df[col] = df[col].fillna(median)
    print(f"{count} valores ausentes na coluna {col} foram substituídos por seu valor de mediana {median}.")
    return df[col]


# Preenche valor NA
def fix_missing_value(df, col, value):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(value)
    if type(value) == 'str':
        print(f"{count} valores ausentes na coluna {col} foram substituídos por '{value}'.")
    else:
        print(f"{count} valores ausentes na coluna {col} foram substituídos por {value}.")
    return df[col]


# Drop duplicatas
def drop_duplicates(df):
    old = df.shape[0]
    df.drop_duplicates(inplace = True)
    new = df.shape[0]
    count = old - new
    if (count == 0):
        print("Nenhuma linha duplicada foi encontrada.")
    else:
        print(f"{count} linhas duplicadas foram encontradas e removidas.")


# Drop de linhas com valores ausentes
def drop_rows_with_missing_values(df):
    old = df.shape[0]
    df.dropna(inplace = True)
    new = df.shape[0]
    count = old - new
    print(f"{count} linhas contendo valores ausentes foram descartadas.")


# Drop de colunas
def drop_columns(df, columns):
    df.drop(columns, axis = 1, inplace = True)
    count = len(columns)
    if count == 1:
        print(f"{count} coluna foi descartada.")
    else:
        print(f"{count} colunas foram descartadas.")





        

Módulo de conversão de tipos de dados

In [ ]:






def convert_to_string(df, columns):
    for col in columns:
        df[col] = df[col].astype("string")


def convert_to_int(df, columns):
    for col in columns:
        df[col] = df[col].astype("int64")


def convert_to_datetime(df, columns):
    for col in columns:
        df[col] = pd.to_datetime(df[col])


def multiply_by_factor(df, columns, factor):
    for col in columns:
        df[col] = df[col] * factor


Módulo de tratamento de outliers

In [ ]:


# Classe
class TrataOutlier:

    # Construtor
    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df

    def count_outliers(self, Q1, Q3, IQR, columns):
        cut_off = IQR * 1.5
        temp_df = (self.df[columns] < (Q1 - cut_off)) | (self.df[columns] > (Q3 + cut_off))
        return [len(temp_df[temp_df[col] == True]) for col in temp_df]

    def calc_skew(self, columns=None):
        if columns == None:
            columns = self.df.columns
        return [self.df[col].skew() for col in columns]

    def percentage(self, list):
        return [str(round(((value/150001) * 100), 2)) + '%' for value in list]

    def remove_outliers(self, columns):
        for col in columns:
            Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            cut_off = IQR * 1.5
            lower, upper = Q1 - cut_off, Q3 + cut_off
            self.df = self.df.drop(self.df[self.df[col] > upper].index)
            self.df = self.df.drop(self.df[self.df[col] < lower].index)

    def replace_outliers_with_fences(self, columns):
        for col in columns:
            Q1, Q3 = self.df[col].quantile(0.25), self.df[col].quantile(0.75)
            IQR = Q3 - Q1
            cut_off = IQR * 1.5
            lower, upper = Q1 - cut_off, Q3 + cut_off

            self.df[col] = np.where(self.df[col] > upper, upper, self.df[col])
            self.df[col] = np.where(self.df[col] < lower, lower, self.df[col])

    def getOverview(self, columns) -> None:
        min = self.df[columns].min()
        Q1 = self.df[columns].quantile(0.25)
        median = self.df[columns].quantile(0.5)
        Q3 = self.df[columns].quantile(0.75)
        max = self.df[columns].max()
        IQR = Q3 - Q1
        skew = self.calc_skew(columns)
        outliers = self.count_outliers(Q1, Q3, IQR, columns)
        cut_off = IQR * 1.5
        lower, upper = Q1 - cut_off, Q3 + cut_off

        new_columns = ['Nome de Coluna', 'Min', 'Q1', 'Median', 'Q3', 'Max', 'IQR', 'Lower fence', 'Upper fence', 'Skew', 'Num_Outliers', 'Percent_Outliers' ]
        
        data = zip([column for column in self.df[columns]], min, Q1, median, Q3, max, IQR, lower, upper, skew, outliers, self.percentage(outliers))

        new_df = pd.DataFrame(data = data, columns = new_columns)
        
        new_df.set_index('Nome de Coluna', inplace = True)
        \
        return new_df.sort_values('Num_Outliers', ascending = False).transpose()




###Carregando os Dados

In [ ]:
# Configurando o número máximo de colunas a serem exibidas

pd.set_option ('display.max_columns', 100)
# Criando lista com possiveis valores NaN

lista_NaN = ["n/a", "na", "undefined"]
# Carregando

dados = '/content/dataset.csv'
dataset = pd.read_csv (dados, na_values = lista_NaN )

In [ ]:
# Shape

dataset.shape

#(Temos 150001 linhas e 55 colunas )



(150001, 55)

In [ ]:
# Visualidando o dataframe

dataset.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,37624.0,38787.0,1.823653e+09,Samsung,Samsung Galaxy A5 Sm-A520F,NaN,NaN,NaN,NaN,NaN,NaN,213.0,214.0,1545765.0,24420.0,1634479.0,1271433.0,3563542.0,137762.0,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,168.0,3560.0,1.365104e+09,Samsung,Samsung Galaxy J5 (Sm-J530),NaN,NaN,NaN,NaN,NaN,NaN,971.0,1022.0,1926113.0,7165.0,3493924.0,920172.0,629046.0,308339.0,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.361763e+09,Samsung,Samsung Galaxy A8 (2018),NaN,NaN,NaN,NaN,NaN,NaN,751.0,695.0,1684053.0,42224.0,8535055.0,1694064.0,2690151.0,672973.0,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,3330.0,37882.0,1.321510e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,207.0,644121.0,13372.0,9023734.0,2788027.0,1439754.0,631229.0,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.089009e+09,Samsung,Samsung Sm-G390F,NaN,NaN,NaN,NaN,NaN,NaN,607.0,604.0,862600.0,50188.0,6248284.0,1500559.0,1936496.0,173853.0,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [ ]:
# Carregando o Dicionario de dados

dic = '/content/Dicionario.xlsx'
dicionario = pd.read_excel ( dic)

In [ ]:
# Visualizando o dicionario

dicionario.head()

,Fields,Description
0,bearer id,xDr session identifier
1,Dur. (ms),Total Duration of the xDR (in ms)
2,Start,Start time of the xDR (first frame timestamp)
3,Start ms,Milliseconds offset of start time for the xDR ...
4,End,End time of the xDR (last frame timestamp)


### Análise Exploratória 

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [ ]:
dataset.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,150000.000000,6.185500e+04,5.335200e+04,149247.000000,149247.000000,149247.000000,149247.000000,149209.000000,149209.000000,149209.000000,149209.000000,6.852700e+04,6.819100e+04,1.500000e+05,1.500000e+05,1.500000e+05,52463.000000,57107.000000,56415.000000,19747.000000,61684.000000,38158.000000,149246.000000,149208.000000,1.500010e+05,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,1770.428647,2.080991e+07,7.596587e+05,92.844754,3.069355,1.717341,1.609654,98.530142,0.776749,0.147987,0.078923,1.144710e+08,3.242301e+06,1.829177e+06,1.408880e+06,1.046091e+08,989.699998,340.434395,810.837401,149.257052,965.464756,141.304812,3719.787552,4022.083454,1.795322e+06,32928.434380,5.750753e+06,2.056542e+06,1.791729e+06,467373.441940,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,4625.355500,1.825665e+08,2.645305e+07,13.038031,6.215233,4.159538,4.828890,4.634285,3.225176,1.624523,1.295396,9.631946e+08,1.957064e+07,5.696395e+06,4.643231e+06,8.103761e+07,2546.524440,1445.365032,1842.162008,1219.112287,1946.387608,993.349688,9171.609010,10160.324314,1.035482e+06,19006.178256,3.309097e+06,1.189917e+06,1.035840e+06,269969.307031,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,0.000000,2.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+01,4.000000e+01,0.000000e+00,0.000000e+00,7.142988e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.200000e+01,0.000000,2.070000e+02,3.000000e+00,1.400000e+01,2.000000,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,47.000000,3.565150e+04,4.694750e+03,91.000000,0.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000,1.124035e+05,2.432200e+04,1.487775e+04,2.153975e+04,5.744079e+07,20.000000,10.000000,26.000000,2.000000,39.000000,3.000000,87.000000,106.000000,8.991480e+05,16448.000000,2.882393e+06,1.024279e+06,8.927930e+05,233383.000000,5.833501e+06,5.517965e+06,5.777156

In [ ]:
# Shape Dataset

dataset.shape

(150001, 55)

In [ ]:
# Shape Dicionario

In [ ]:
dicionario.shape

(56, 2)

Há uma incongruencia aqui. O numero de colunas do dataset não bate com os descritor de colunas do dicionario

In [ ]:
# Concatenando os dataframes

df_compara_colunas = pd.concat([pd.Series( dataset.columns.tolist()),
                                dicionario ['Fields']], axis = 1 )

In [ ]:
#Visualizando o novo dataframe
df_compara_colunas


,0,Fields
0,Bearer Id,bearer id
1,Start,Dur. (ms)
2,Start ms,Start
3,End,Start ms
4,End ms,End
5,Dur. (ms),End ms
6,IMSI,Dur. (s)
7,MSISDN/Number,IMSI
8,IMEI,MSISDN/Number
9,Last Location Name,IMEI


In [ ]:
#Renomeando as Colunas
df_compara_colunas.rename(columns = {0: "Coluna do Dataset", "Fields": "Coluna do Dicionario"}, inplace= True)
df_compara_colunas

,Coluna do Dataset,Coluna do Dicionario
0,Bearer Id,bearer id
1,Start,Dur. (ms)
2,Start ms,Start
3,End,Start ms
4,End ms,End
5,Dur. (ms),End ms
6,IMSI,Dur. (s)
7,MSISDN/Number,IMSI
8,IMEI,MSISDN/Number
9,Last Location Name,IMEI


Vemos que algumas colunas do dataset estão erradas se comparada com o dicionario de dados, iremos concertar isso!

In [ ]:
#Renomeando as colunas no dataset

dataset.rename(columns = {"Dur. (ms)":"Dur. (s)",
                          "Dur. (ms).1": "Dur. (ms)"
                          },
               inplace = True)


In [ ]:
dataset.columns.tolist()

['Bearer Id',
 'Start',
 'Start ms',
 'End',
 'End ms',
 'Dur. (s)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms)',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Social M

In [ ]:
#Verificando o percent de valores ausentes do dataset

func_calc_percentual_valores_ausentes(dataset)

O dataset tem 12.72 % de valores ausentes.


In [ ]:
#Verificando a quantidade de colunas que possui ao menos um valor ausente

dataset_valores_ausentes = func_calc_percentual_valores_ausentes_coluna (dataset)

O dataset tem 55 colunas.
Encontrado: 41 colunas que têm valores ausentes.


In [ ]:
#Verificando a tabela com os valores ausentes
dataset_valores_ausentes

,Valores Ausentes,% de Valores Ausentes,Dtype
Nb of sec with 37500B < Vol UL,130254,86.84,float64
Nb of sec with 6250B < Vol UL < 37500B,111843,74.56,float64
Nb of sec with 125000B < Vol DL,97538,65.02,float64
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
Nb of sec with 31250B < Vol DL < 125000B,93586,62.39,float64
Nb of sec with 1250B < Vol UL < 6250B,92894,61.93,float64
Nb of sec with 6250B < Vol DL < 31250B,88317,58.88,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
HTTP UL (Bytes),81810,54.54,float64
HTTP DL (Bytes),81474,54.32,float64


In [ ]:
# Coluas que serão removidas

colunas_remover = dataset_valores_ausentes [dataset_valores_ausentes [ '% de Valores Ausentes']>= 30.00].index.tolist()
colunas_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'TCP UL Retrans. Vol (Bytes)',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'TCP DL Retrans. Vol (Bytes)',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

Mesmo que as variáveis TCP tenham muitos NaN, iremos imputar dados nessas colunas. Por se tratar de uma variável importante no dataset.

In [ ]:
#Colunas que realmente serão removidas
colunas_remover = [col for col in colunas_remover if col not in ['TCP UL Retrans. Vol (Bytes)', 'TCP DL Retrans. Vol (Bytes)']]
colunas_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [ ]:
#Dropando as coulunas
dataset_limpo = dataset.drop (colunas_remover, axis = 1)

In [ ]:
#De 55 Colunas, ficamos com 47
dataset_limpo.shape

(150001, 47)

In [ ]:
#Verificando o percent de valores ausentes do dataset por coluna

dataset_valores_ausentes = func_calc_percentual_valores_ausentes_coluna (dataset_limpo)
dataset_valores_ausentes

O dataset tem 47 colunas.
Encontrado: 33 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
Avg RTT DL (ms),27829,18.55,float64
Avg RTT UL (ms),27812,18.54,float64
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64


In [ ]:
#Iremos aplicar o preenchimento reverso nas variaveis TCP UL Retrans. Vol (Bytes) e TCP DL Retrans. Vol (Bytes)

fix_missing_bfill(dataset_limpo,'TCP UL Retrans. Vol (Bytes)')

96649 valores ausentes na coluna TCP UL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         7230.0
1         7230.0
2         7230.0
3         7230.0
4         7230.0
           ...  
149996       NaN
149997       NaN
149998       NaN
149999       NaN
150000       NaN
Name: TCP UL Retrans. Vol (Bytes), Length: 150001, dtype: float64

In [ ]:
fix_missing_bfill(dataset_limpo,'TCP DL Retrans. Vol (Bytes)')

88146 valores ausentes na coluna TCP DL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         19520.0
1         19520.0
2         19520.0
3         19520.0
4         19520.0
           ...   
149996        NaN
149997        NaN
149998        NaN
149999        NaN
150000        NaN
Name: TCP DL Retrans. Vol (Bytes), Length: 150001, dtype: float64

In [ ]:
# Verificando se as variaveis Avg RTT DL (ms) e Avg RTT UL (ms) 
# seguem uma distribuição normal, se sim iremos utilizar a mediana como valor a ser imputado, se não iremos utilizar o preenchimento progressivo.

dataset_limpo['Avg RTT DL (ms)'].skew(skipna = True)


62.90782807995961

In [ ]:
dataset_limpo['Avg RTT UL (ms)'].skew(skipna = True)

28.45741458546382

Como os valores foram acima de 1, os dados estão altamente enviesados e não seguem uma distribuição normal, logo iremos utilizar o metódo de preenchimento progressivo.

In [ ]:
# Imputando com preenchimento progressivo

fix_missing_ffill(dataset_limpo,'Avg RTT DL (ms)')

27829 valores ausentes na coluna Avg RTT DL (ms) foram substituídos usando o método de preenchimento progressivo.


0         42.0
1         65.0
2         65.0
3         65.0
4         65.0
          ... 
149996    32.0
149997    27.0
149998    43.0
149999    37.0
150000    37.0
Name: Avg RTT DL (ms), Length: 150001, dtype: float64

In [ ]:
fix_missing_ffill(dataset_limpo,'Avg RTT UL (ms)')

27812 valores ausentes na coluna Avg RTT UL (ms) foram substituídos usando o método de preenchimento progressivo.


0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
         ... 
149996    0.0
149997    2.0
149998    6.0
149999    5.0
150000    5.0
Name: Avg RTT UL (ms), Length: 150001, dtype: float64

In [ ]:
#Verificando o percent de valores ausentes do dataset

func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 0.44 % de valores ausentes.


In [ ]:
# Tratamento de valores ausentes das variaveis categoricas : Handset Manufacturer e Last Location Name

fix_missing_value(dataset_limpo, 'Handset Manufacturer', 'Other')


9559 valores ausentes na coluna Handset Manufacturer foram substituídos por Other.


0         Samsung
1         Samsung
2         Samsung
3           Other
4         Samsung
           ...   
149996      Apple
149997      Apple
149998      Apple
149999     Huawei
150000      Other
Name: Handset Manufacturer, Length: 150001, dtype: object

In [ ]:
fix_missing_value(dataset_limpo, 'Last Location Name', 'Other')

1153 valores ausentes na coluna Last Location Name foram substituídos por Other.


0         9.16456699548519E+015
1                       L77566A
2                       D42335A
3                       T21824A
4                       D88865A
                  ...          
149996                  D20434A
149997                  D10223C
149998                  T51102A
149999                  L88342B
150000                    Other
Name: Last Location Name, Length: 150001, dtype: object

In [ ]:
#Verificando o percent de valores ausentes do dataset

func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 0.29 % de valores ausentes.


In [ ]:
# Calculando o total de valores ausentes por linha do dataset

func_calc_percentual_valores_ausentes_linha(dataset_limpo)

7.87 % das linhas no conjunto de dados contêm pelo menos um valor ausente.


In [ ]:
# Dropando os restandes das linhas com valores ausentes

drop_rows_with_missing_values(dataset_limpo)

11812 linhas contendo valores ausentes foram descartadas.


In [ ]:
#Verificando o percent de valores ausentes do dataset

func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 0.0 % de valores ausentes.


Pronto Dataset completamente tratado os seus valores ausentes!

### Convertendo os Tipos de Dados

In [ ]:
#Verificando os tipos de dados do df

dataset_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138189 entries, 0 to 149994
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       138189 non-null  int64         
 1   Start                           138189 non-null  datetime64[ns]
 2   Start ms                        138189 non-null  float64       
 3   End                             138189 non-null  datetime64[ns]
 4   End ms                          138189 non-null  float64       
 5   Dur. (s)                        138189 non-null  float64       
 6   IMSI                            138189 non-null  int64         
 7   MSISDN/Number                   138189 non-null  int64         
 8   IMEI                            138189 non-null  int64         
 9   Last Location Name              138189 non-null  string        
 10  Avg RTT DL (ms)                 138189 non-null  float64

In [ ]:
# Convertendo as variáveis para datetime

convert_to_datetime(dataset_limpo, ['Start', 'End'])

In [ ]:
# Extraindo as colunas do tipo object para convertermos para str

colunas_str = dataset_limpo.select_dtypes(include = 'object').columns.tolist()

colunas_str

['Last Location Name', 'Handset Manufacturer', 'Handset Type']

In [ ]:
# Convertendo as variaveis ['Last Location Name', 'Handset Manufacturer', 'Handset Type'] para STR

convert_to_string(dataset_limpo,colunas_str)

In [ ]:
# Convertendo as variaveis de ID para INT
colunas_int = ['Bearer Id', 'IMSI', 'MSISDN/Number', 'IMEI']
convert_to_int(dataset_limpo,colunas_int)

In [ ]:
#Verificando os tipos de dados do df

dataset_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138189 entries, 0 to 149994
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       138189 non-null  int64         
 1   Start                           138189 non-null  datetime64[ns]
 2   Start ms                        138189 non-null  float64       
 3   End                             138189 non-null  datetime64[ns]
 4   End ms                          138189 non-null  float64       
 5   Dur. (s)                        138189 non-null  float64       
 6   IMSI                            138189 non-null  int64         
 7   MSISDN/Number                   138189 non-null  int64         
 8   IMEI                            138189 non-null  int64         
 9   Last Location Name              138189 non-null  string        
 10  Avg RTT DL (ms)                 138189 non-null  float64

### Tratamento dos Outliers 

In [ ]:
# Cria o objeto trata outlier
trata_outlier = TrataOutlier(dataset_limpo)


In [ ]:
# Lista de colunas float64
lista_colunas = dataset_limpo.select_dtypes('float64').columns.tolist()
lista_colunas

['Start ms',
 'End ms',
 'Dur. (s)',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms)',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Social Media DL (Bytes)',
 'Social Media UL (Bytes)',
 'Google DL (Bytes)',
 'Google UL (Bytes)',
 'Email DL (Bytes)',
 'Email UL (Bytes)',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Netflix DL (Bytes)',
 'Netflix UL (Bytes)',
 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)',
 'Total UL (Bytes)',
 'Total DL (Bytes)']

In [ ]:
# Visão geral dos outliers
trata_outlier.getOverview(lista_colunas)

Nome de Coluna,10 Kbps < UL TP < 50 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),Activity Duration DL (ms),Activity Duration UL (ms),Nb of sec with Vol UL < 1250B,Nb of sec with Vol DL < 6250B,DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),TCP DL Retrans. Vol (Bytes),Avg Bearer TP UL (kbps),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Dur. (ms),Dur. (s),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),Total UL (Bytes),Youtube UL (Bytes),Other UL (Bytes),Other DL (Bytes),Gaming UL (Bytes),Gaming DL (Bytes),Netflix UL (Bytes),Netflix DL (Bytes),Start ms,Youtube DL (Bytes),Email UL (Bytes),Email DL (Bytes),Google UL (Bytes),Google DL (Bytes),Social Media UL (Bytes),Social Media DL (Bytes),End ms,Total DL (Bytes)
Min,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7142988.0,7142.0,0.0,0.0,2866892.0,105.0,148.0,14320.0,59.0,2516.0,35.0,42.0,0.0,53.0,2.0,14.0,3.0,207.0,0.0,12.0,0.0,7114041.0
Q1,0.0,0.0,16156.0,22455.0,109.0,90.0,0.0,99.0,25524.0,47.0,3439.0,31.0,90.0,0.0,2.0,43.0,57786839.0,57786.0,0.0,0.0,33212508.0,5514946.0,4152525.0,210447006.0,4133444.0,209930291.0,5466972.0,5781857.0,250.0,5817869.0,233123.0,893194.0,1024694.0,2878717.0,16427.0,897896.0,251.0,242620333.0
Median,0.0,0.0,41417.0,48359.0,224.0,209.0,0.0,100.0,406912.0,64.0,16744.0,44.0,100.0,0.0,5.0,65.0,86399981.0,86399.0,0.0,0.0,41134201.0,11017754.0,8267249.0,421894145.0,8291640.0,422900534.0,10993807.0,11640622.0,499.0,11613009.0,465941.0,1793105.0,2055233.0,5770074.0,32911.0,1794506.0,500.0,455231716.0
Q3,0.0,1.0,766736.0,671852.0,2679.0,2873.0,1.0,100.0,3145260.0,1191.0,69405.0,67.0,100.0,4.0,14.0,20889.0,130642965.0,130642.0,0.0,0.0,49030528.0,16513028.0,12376673.0,631671704.0,12428987.0,633051900.0,16501438.0,17467338.0,749.0,17448055.0,699977.0,2688526.0,3088454.0,8627489.0,49313.0,2693880.0,750.0,665575131.0
Max,98.0,76.0,136536461.0,144911293.0,604122.0,604061.0,94.0,100.0,4294425570.0,58613.0,2908226006.0,96923.0,100.0,93.0,7120.0,378160.0,1859336442.0,1859336.0,85.0,96.0,78331311.0,22011962.0,16558816.0,843442489.0,16558794.0,843441889.0,22011955.0,23259189.0,999.0,23259098.0,936418.0,3586146.0,4121357.0,11462832.0,65870.0,3586064.0,999.0,902969616.0
IQR,0.0,1.0,750580.0,649397.0,2570.0,2783.0,1.0,1.0,3119736.0,1144.0,65966.0,36.0,10.0,4.0,12.0,20846.0,72856126.0,72856.0,0.0,0.0,15818020.0,10998082.0,8224148.0,421224698.0,8295543.0,423121609.0,11034466.0,11685481.0,499.0,11630186.0,466854.0,1795332.0,2063760.0,5748772.0,32886.0,1795984.0,499.0,422954798.0
Lower fence,0.0,-1.5,-1109714.0,-951640.5,-3746.0,-4084.5,-1.5,97.5,-4654080.0,-1669.0,-95510.0,-23.0,75.0,-6.0,-16.0,-31226.0,-51497350.0,-51498.0,0.0,0.0,9485478.0,-10982177.0,-8183697.0,-421390041.0,-8309870.5,-424752122.5,-11084727.0,-11746364.5,-498.5,-11627410.0,-467158.0,-1799804.0,-2070946.0,-5744441.0,-32902.0,-1796080.0,-497.5,-391811864.0
Upper fence,0.0,2.5,1892606.0,1645947.5,6534.0,7047.5,2.5,101.5,7824864.0,2907.0,168354.0,121.0,115.0,10.0,32.0,52158.0,239927154.0,239926.0,0.0,0.0,72757558.0,33010151.0,24712895.0,1263508751.0,24872301.5,1267734313.5,33053137.0,34995559.5,1497.5,34893334.0,1400258.0,5381524.0,6184094.0,17250647.0,98642.0,5387856.0,1498.5,1300007328.0
Skew,10.773428,4.447445,5.666114,7.139486,7.313969,8.827921,5.237783,-8.821305,16.563721,4.391731,93.498542,67.2808,-2.260558,3.263605,27.419359,2.525679,3.883689,3.883689,20.63609,35.033422,-0.001345,-0.001519,0.001372,0.001724,-0.002009,-0.002277,0.000387,-0.001808,0.000948,0.000477,0.007886,-0.002168,0.002198,-0.008654,0.000199,-0.002817,-0.00213,-0.001937
Num_Outliers,30226,28329,24404,23635,23248,22858,21951,20982,20415,19940,19283,15483,14703,14246,13492,11048,6773,6773,4366,2369,224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Replace dos outliers
trata_outlier.replace_outliers_with_fences(lista_colunas)

In [ ]:
# Visão geral dos outliers
trata_outlier.getOverview(lista_colunas)

Nome de Coluna,Start ms,Youtube DL (Bytes),Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube UL (Bytes),End ms,Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Nb of sec with Vol DL < 6250B,Dur. (ms),Activity Duration UL (ms),Activity Duration DL (ms),Dur. (s),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),Total DL (Bytes)
Min,0.0,53.0,1.0,12.0,0.0,207.0,3.0,14.0,2.0,105.0,0.0,42.0,35.0,2516.0,59.0,14320.0,148.0,9485478.0,1.0,7142988.0,0.0,0.0,7142.0,0.0,0.0,0.0,0.0,2.0,1.0,75.0,0.0,0.0,0.0,97.5,0.0,0.0,0.0,7114041.0
Q1,250.0,5817869.0,109.0,897896.0,16427.0,2878717.0,1024694.0,893194.0,233123.0,5514946.0,251.0,5781857.0,5466972.0,209930291.0,4133444.0,210447006.0,4152525.0,33212508.0,90.0,57786839.0,22455.0,16156.0,57786.0,31.0,2.0,43.0,47.0,25524.0,3439.0,90.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,242620333.0
Median,499.0,11613009.0,224.0,1794506.0,32911.0,5770074.0,2055233.0,1793105.0,465941.0,11017754.0,500.0,11640622.0,10993807.0,422900534.0,8291640.0,421894145.0,8267249.0,41134201.0,209.0,86399981.0,48359.0,41417.0,86399.0,44.0,5.0,65.0,64.0,406912.0,16744.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,455231716.0
Q3,749.0,17448055.0,2679.0,2693880.0,49313.0,8627489.0,3088454.0,2688526.0,699977.0,16513028.0,750.0,17467338.0,16501438.0,633051900.0,12428987.0,631671704.0,12376673.0,49030528.0,2873.0,130642965.0,671852.0,766736.0,130642.0,67.0,14.0,20889.0,1191.0,3145260.0,69405.0,100.0,4.0,1.0,1.0,100.0,0.0,0.0,0.0,665575131.0
Max,999.0,23259098.0,6534.0,3586064.0,65870.0,11462832.0,4121357.0,3586146.0,936418.0,22011962.0,999.0,23259189.0,22011955.0,843441889.0,16558794.0,843442489.0,16558816.0,72757558.0,7047.5,239927154.0,1645947.5,1892606.0,239926.0,121.0,32.0,52158.0,2907.0,7824864.0,168354.0,100.0,10.0,2.5,2.5,100.0,0.0,0.0,0.0,902969616.0
IQR,499.0,11630186.0,2570.0,1795984.0,32886.0,5748772.0,2063760.0,1795332.0,466854.0,10998082.0,499.0,11685481.0,11034466.0,423121609.0,8295543.0,421224698.0,8224148.0,15818020.0,2783.0,72856126.0,649397.0,750580.0,72856.0,36.0,12.0,20846.0,1144.0,3119736.0,65966.0,10.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,422954798.0
Lower fence,-498.5,-11627410.0,-3746.0,-1796080.0,-32902.0,-5744441.0,-2070946.0,-1799804.0,-467158.0,-10982177.0,-497.5,-11746364.5,-11084727.0,-424752122.5,-8309870.5,-421390041.0,-8183697.0,9485478.0,-4084.5,-51497350.0,-951640.5,-1109714.0,-51498.0,-23.0,-16.0,-31226.0,-1669.0,-4654080.0,-95510.0,75.0,-6.0,-1.5,-1.5,97.5,0.0,0.0,0.0,-391811864.0
Upper fence,1497.5,34893334.0,6534.0,5387856.0,98642.0,17250647.0,6184094.0,5381524.0,1400258.0,33010151.0,1498.5,34995559.5,33053137.0,1267734313.5,24872301.5,1263508751.0,24712895.0,72757558.0,7047.5,239927154.0,1645947.5,1892606.0,239926.0,121.0,32.0,52158.0,2907.0,7824864.0,168354.0,115.0,10.0,2.5,2.5,101.5,0.0,0.0,0.0,1300007328.0
Skew,0.000948,0.000477,1.209635,-0.002817,0.000199,-0.008654,0.002198,-0.002168,0.007886,-0.001519,-0.00213,-0.001808,0.000387,-0.002277,-0.002009,0.001724,0.001372,-0.001436,1.2072,0.793087,1.211229,1.205772,0.793084,1.050112,1.21421,1.281787,1.227272,1.214029,1.195979,-1.273549,1.28712,1.061914,1.38291,-0.918125,0.0,0.0,0.0,-0.001937
Num_Outliers,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Salvando o projeto

In [ ]:
# Salvando o projeto no formato CSV
dataset_limpo.to_csv('/content/dataset_limpo.csv')

###Fim